# Some Abstract Ideas

## Abstract Number Types

### SVectors

Say we're using a really simple time stepper to update the velocity of the particles stored in a vector `v`. To start out, we could initialize the vector `v` as:

In [ ]:
v = zeros(Float64, 3, nparticles)
dvdt = rand(3,3)
dt = 0.5

Then the time stepper could be written as:

In [ ]:
for i in eachindex(v)
  v[1, i] += dvdt[1] #X coordinate of velocity
  v[2, i] += dvdt[2] #Y coordinate of velocity
  v[3, i] += dvdt[3] #Z coordinate of velocity
end

We could collapse this with another for-loop like

In [ ]:
for i in eachindex(v)
  for pos in 1:3
  v[pos, i] += dvdt[pos] #X coordinate of velocity
end

We can do better. Suppose instead we initialize our array of velocities as precisely that: an array of velocity vectors. Note that we are using SVectors so the compiler will make our tiny loops unroll and be fast.
We would initialize as

In [ ]:
using StaticArrays
v = zeros(SVector{3, Float64}, 3, nparticles)
dvdt = SVector{3, Float64}(rand(3))
dt = 0.5

Then we can write our loop as

In [ ]:
for i in eachindex(v)
  v[i] += dvdt * dt #forward euler step of velocity
end

This works because `+` is defined on two vector types and `*` is defined between scalar and vector types! Way cool!

### Unitful
Probably not good for the undimensionalized innards of clima, but really great for user interfaces before dedimensionalization and for conversion between units!

(see https://github.com/ajkeller34/Unitful.jl)

In [ ]:
julia> 1u"kg" == 1000u"g"             # Equivalence implies unit conversion
true

julia> !(1u"kg" === 1000u"g")         # ...and yet we can distinguish these...
true

julia> 1u"kg" === 1u"kg"              # ...and these are indistinguishable.
true

### Finite Elements
(see https://github.com/JuliaFEM/FEMBasis.jl)

In [ ]:
struct GaussLobattoElement{D#=Number of dimensions=#, N#=Order of Polynomials=#}
    points::SArray
end

function Base.:+(a::GaussLobattoElement{D, N}, b::GaussLobattoElement{D, N}) where {D, N}
    GaussLobattoElement(a.points .+ b.points)
end

function Base.:*(a::GaussLobattoElement, s::Number)
    GaussLobattoElement(a.points .* s)
end

function deriv(a::GaussLobattoElement)
    #maybe some more complicated stuff here involving test/trial functions
end

### States (Separate Space and Time)
(see https://github.com/JuliaDiffEq/SimpleDiffEq.jl, https://github.com/JuliaDiffEq/RecursiveArrayTools.jl, http://www.stochasticlifestyle.com/solving-systems-stochastic-pdes-using-gpus-julia/)

The big idea behind the DiffEquations.jl interface is that we can separate space discretization from time discretization by considering the stepped state of the simulation as a number itself.

![Image Test](https://wikimedia.org/api/rest_v1/media/math/render/svg/81398071e8f180ac143bfdf4598ff47bf79eb689)
![Image Test](https://wikimedia.org/api/rest_v1/media/math/render/svg/95bdbb2e3aa83735085c0aadd577162c69e4056a)

In [ ]:
struct DycoreState
    ρ
    U
    E
end

Base.:+(a::DycoreState, b::DycoreState) = DycoreState(a.ρ + b.ρ, a.U + b.U, a.E + b.E)

Base.:*(a::DycoreState, s::Number) = DycoreState(a.ρ * s, a.U * s, a.E * s)

Base.LinearAlgebra.norm(a::DycoreState) = norm(a.ρ) + norm(a.U) + norm(a.E) #I don't know how math works

f(t, state::DycoreState, params)::DycoreState = #math skills still lacking... Navier Stokes or something here


#Now we can time step without thinking about what a state is!


function step_euler(f, state, t, dt, params)
    return state + dt * f(t, state, params)
end
    
function step_rk(f, state, global_parameters, dt, T_final)
    k1 = dt * f(t,        state,        params)
    k2 = dt * f(t + dt/2, state + k1/2, params)
    k3 = dt * f(t + dt/2, state + k2/2, params)
    k4 = dt * f(t + dt,   state + k,    params)
    return state + 1/6(k1 + 2k2 + 3k3 + k4)
    
end


# Fields

In [ ]:
type AbstractField{T, D} end

struct ConstantField{T, D}
    val::T
end

get_value(f::ConstantField{T, D}, x:::NTuple{D}) where {T, D} = f.val


struct DeformedField{T, D, X::Field{NTuple{D}, D}, F<:Field{T, D}}
    ξ::X
    parent::F
end

get_value(f::DeformedField{T, D}, x::Vararg{Any, D}) where {T, D} = get_value(f.parent, f.ξ(x))

function Base.:+(a::DeformedField, b::DeformedField)
    @assert a.ξ == b.ξ
    DeformedField(a.ξ, a.parent + b.parent)
end

function Base.:+(a::DeformedField, s::Number)
    DeformedField(a.ξ, a.parent * s)
end

### Fields over Grids

In [ ]:
struct CartesianGrid{D, E<:CartesianIndices}
    elems::E
end

get_elem(CartesianGrid, x) = CartesianIndex(map(Int, map(floor, x)))

struct GridField{T, D, F::AbstractArray{<:Field{T, D}}, G<:Grid{D}}
    subfields::F
    grid::G
end

get_value(f::GridField, x) = get_value(f.subfields[get_elem(f.grid, x)], x)



function Base.:+(a::GridField, b::GridField)
    @assert a.grid == b.grid
    GridField(a.subfields .+ b.subfields, a.grid)
end

function Base.:*(a::GridField, s::Number)
    GridField(a.subfields .* s, a.grid)
end

# A couple main loops to spark conversation

### GCM

In [ ]:
C     >>>>>>>>>>>>>>>>>>>>>>>>>>>   LOOP   <<<<<<<<<<<<<<<<<<<<<<<<<<<<
C     >>>>>>>>>>>>>>>>>>>>>>>>>>>  STARTS  <<<<<<<<<<<<<<<<<<<<<<<<<<<<
# ifdef ALLOW_OPENAD_DIVA
      DO iloop = 1, nTimeSteps_l2
# else
      DO iloop = 1, nTimeSteps
# endif

#endif /* ALLOW_OPENAD */

#ifdef ALLOW_DEBUG
      IF (debugMode) CALL DEBUG_CALL('FORWARD_STEP',myThid)
#endif

#ifndef ALLOW_OPENAD
# ifdef ALLOW_ATM2D
        CALL TIMER_START('FORWARD_STEP_ATM2D  [MAIN_DO_LOOP]',myThid)
        CALL FORWARD_STEP_ATM2D( iloop, myTime, myIter, myThid )
        CALL TIMER_STOP ('FORWARD_STEP_ATM2D  [MAIN_DO_LOOP]',myThid)
# else
        CALL TIMER_START('FORWARD_STEP        [MAIN_DO_LOOP]',myThid)
        CALL FORWARD_STEP( iloop, myTime, myIter, myThid )
        CALL TIMER_STOP ('FORWARD_STEP        [MAIN_DO_LOOP]',myThid)
# endif
#else
# ifdef ALLOW_OPENAD_DIVA
       CALL TIMER_START('INNER_DO_LOOP       [MAIN_DO_LOOP]',myThid)
       nTimeSteps_l2 = 2
       CALL INNER_DO_LOOP( iloop, myTime, myIter, myThid )
       CALL TIMER_STOP ('INNER_DO_LOOP       [MAIN_DO_LOOP]',myThid)
# else
       CALL TIMER_START('FORWARD_STEP        [MAIN_DO_LOOP]',myThid)
       nTimeSteps_l2 = 2
       CALL FORWARD_STEP( iloop, myTime, myIter, myThid )
       CALL TIMER_STOP ('FORWARD_STEP        [MAIN_DO_LOOP]',myThid)
# endif
#endif /* ndef ALLOW_OPENAD */

#ifdef ALLOW_OPENAD
      ENDDO
C     >>>>>>>>>>>>>>>>>>>>>>>>>>>   LOOP   <<<<<<<<<<<<<<<<<<<<<<<<<<<<
C     >>>>>>>>>>>>>>>>>>>>>>>>>>>  STOPS   <<<<<<<<<<<<<<<<<<<<<<<<<<<<
#endif /* ALLOW_OPENAD */

### Charlie's personal pseudocode

In [ ]:
clima.paramlist = CLIMA.init_parameters()
clima.dycore = DyCore.init()
N_x, N_y = clima.dycore.N_x, clima.dycore.N_y
clima.turb_conv = [TurbConv.init(clima.paramlist) for i in 1:N_x for j in 1:N_y]
for i in 1:clima.n_steps
 # computes RHS terms (dycore_sources) for the dycore
 compute_sources.(clima.turb_conv,
                 clima.dycore.Q[:u],
                 clima.dycore.Q[:v],
                 clima.dycore.Q[:w],
                 clima.dycore.Q[:energy],
                 ...
                 clima.paramlist)
 for v in (:u, :v, :w, :energy, ...)
   dycore.RHS[i,j,k,v] += interp(
       clima.turb_conv[i,j].dycore_sources[v],
       clima.dycore.mesh)
 end
 # updates state vector dycore.Q from dycore.RHS
 update_solution(clima.dycore)
end

### Main Loop with Subdivisions

In [ ]:
struct Atmosphere
    core
    tracers
end

struct AtmosCore{N}
    ρ::AbstractField
    U::AbstractField{<:SVector{N}}
    E::AbstractField
    subscale
end

struct Ocean
    core::OceanCore
    tracers::Tracers
end

mutable struct Steppable
    state
    stepper
    dt
    t
    callbacks
end

function update!(s::Steppable, t)
    for i in 1:ceil((t - t(state)) / dt(state))
        s.state = s.stepper(s.state, dt)
        do_callbacks(state, niters(state))
        do_other_interesting_per_loop_stuff
    end
end

function step!(atm::Atmosphere, dt)
    update!(atm.core, dt(clm))
    update!(atm.tracers, atm.core, dt(clm))
end
    
function step!(atm::AtmosCore, dt)
    #update the atmospheric core
    update!(atm.subscale, dt)
    atm.core.rhs += rhs(atm.subscale)
end

### Dycore Control

In [ ]:
runner = AD.Runner(mpicomm,
                   #Space Discretization and Parameters
                   :VanillaEuler,
                   (DFloat = DFloat,
                    DeviceArray = backend,
                    meshgenerator = (part, numparts) ->
                    meshgenerator(part, numparts, Ne, dim,
                                  DFloat),
                    dim = dim,
                    gravity = false,
                    N = N,
                   ),
                   # Time Discretization and Parameters
                   :LSRK,
                   (),
                  )

# Set the initial condition with a function
AD.initspacestate!(runner, host=true) do (x...)
  isentropicvortex(DFloat(timeinitial), x...)
end

base_dt = 1e-3
nsteps = ceil(Int64, timeend / base_dt)
dt = timeend / nsteps

# Set the time step
AD.inittimestate!(runner, dt)

eng0 = AD.L2solutionnorm(runner; host=true)
# mpirank == 0 && @show eng0

cberr =
  AD.GenericCallbacks.EveryXWallTimeSeconds(2, mpicomm) do
    err = AD.L2errornorm(runner, isentropicvortex; host=true)
    println(io, "VanillaEuler with errnorm2(Q) = ", err, " at time = ",
            AD.solutiontime(runner))
  end

AD.run!(runner; numberofsteps=nsteps, callbacks=(cbinfo, cbvtk,
                                                 cberr))